<center><img src="http://sydney.edu.au/images/content/about/logo-mono.jpg"></center>

<center><h1>Statistical Learning and Data Mining (QBUS6810)</h1></center>
<center><h2>Tutorial 11: Model Stacking</h2></center>
<br>

In this notebook we build on the analysis of the California housing dataset to consider model averaging. 

<a class="buttom" href=">#Data:-California-Housing">Data: California Housing</a> <br>
<a class="buttom" href="#Previous-results">Previous results</a> <br>
<a class="buttom" href="#Adding-new-models">Generalised additive modelling</a> <br>
<a class="buttom" href="#Model-stacking">Model stacking</a> <br>
<a class="buttom" href="#Model-evaluation">Model Evaluation</a> <br>

This notebook relies on the following libraries and settings.

In [1]:
# Packages
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [2]:
# Methods
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from statlearning import generalised_additive_regression
from statlearning import rmse_jack, r2_jack, mae

##Data processing

In [3]:
data = pd.read_csv('cal_housing.csv')
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
data=data[data['AveRooms']<data['AveRooms'].quantile(.99)]
data=data[data['Population']<data['Population'].quantile(.99)]
data=data[data['AveOccup']<data['AveOccup'].quantile(.99)]

response = data.columns[-1]
predictors= list(data.columns[:-1])
    
train = data.sample(frac=0.2, random_state=1)
test = data[data.index.isin(train.index)==False].copy()

y_train = np.log(train[response])
y_test = np.log(test[response])
X_train = train[predictors]
X_test = test[predictors]

X_train_tree = X_train.copy()
X_test_tree = X_test.copy()

mu=X_train.mean()
sigma=X_train.std()

X_train=(X_train-mu)/sigma
X_test=(X_test-mu)/sigma

##Previous results

This selection replicates the results of the previous tutorial, where we found that gradient boosting was the most accurate method for the test data among a range of candidates including a linear regression, a generalised additive model based on regression splines, and a random forest. 

In [5]:
# OLS
ols = LinearRegression()
ols.fit(X_train, y_train)

# Lasso
lasso = LassoCV(cv=5)
lasso.fit(X_train, y_train)

# Ridge
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(X_train, y_train)

# Decision Tree
tree = DecisionTreeRegressor(max_depth=9, min_samples_leaf=10)
tree.fit(X_train_tree, y_train)

# Bagged trees
bag = BaggingRegressor(n_estimators=1000, random_state=1)
bag.fit(X_train_tree, y_train)

# Random forest
rf = RandomForestRegressor(n_estimators=5000, max_features = 3, min_samples_leaf= 1)
rf.fit(X_train_tree, y_train)

# Gradient boosting
gb = GradientBoostingRegressor(learning_rate= 0.05, max_depth = 5, n_estimators= 750)
gb.fit(X_train_tree, y_train)

# Regression splines 
nonlinear=['MedInc', 'HouseAge', 'AveOccup', 'AveRooms', 'Population', 'AveBedrms', 'Latitude', 'Longitude']
dfs=[8,8,5,3,2,6,18,18]
spline_dfs=dict(zip(nonlinear,dfs))

gam = generalised_additive_regression()
gam.fit(X_train, y_train, spline_dfs)

In [6]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE', 'Test MAE', 'SE']
rows=['OLS', 'Ridge', 'Lasso', 'GAM', 'Decision tree', 'Bagged trees', 'Random forest', 'Gradient boosting']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

methods=[ols, ridge, lasso, gam, tree, bag, rf, gb]

for i, method in enumerate(methods):
    if i < 4:
        y_pred=method.predict(X_test)
    else:
        y_pred=method.predict(X_test_tree)
    
    results.iloc[i,0], results.iloc[i,1] = rmse_jack(y_test, y_pred)
    results.iloc[i,2], results.iloc[i,3] = r2_jack(y_test, y_pred)
    results.iloc[i,4], results.iloc[i,5] = mae(y_test, y_pred)

results.round(3)

,Test RMSE,SE,Test R2,SE,Test MAE,SE
OLS,0.324,0.003,0.676,0.005,0.244,0.002
Ridge,0.324,0.003,0.676,0.005,0.244,0.002
Lasso,0.324,0.003,0.676,0.005,0.244,0.002
GAM,0.267,0.003,0.779,0.004,0.196,0.001
Decision tree,0.313,0.003,0.697,0.006,0.225,0.002
Bagged trees,0.255,0.003,0.799,0.004,0.181,0.001
Random forest,0.249,0.003,0.808,0.004,0.179,0.001
Gradient boosting,0.232,0.003,0.833,0.004,0.164,0.001


##Generalised additive modelling 

In order to obtain accuracy gains from model stacking, we diversify our range of methods to consider a more tailored generalised additive model (GAM). Our GAM will combine individual regression splines for most with a local regression for the spatial effect.

The model has the form: 

\begin{equation}
Y=\beta_0+\left(\sum_{j=1}^{p-2}\,f_j(X_j)\right)+g\,(\text{latitude},\text{longitude})+\varepsilon.
\end{equation}

We will estimate $f_j(\cdot)$ with natural splines and $g(\cdot,)$ with a kernel weighted average (local constant model). The flexibility of a nonparametric regression is well suited for estimating the spatial effects.    

To illustrate the use of local regression as building block, the next cell estimates a local linear regression with three predictors:  median income, latitude, and longitude. 

In [7]:
%%time
from statsmodels.nonparametric.kernel_regression import KernelReg
kernel = KernelReg(y_train, X_train.iloc[:,[0,6,7]], var_type=3*'c', reg_type='ll')

Wall time: 41min 8s


While a local regression can be very flexible, it is has the disadvantages of being (i) subject to the curse of dimensionality, which we mitigate by considering only a few predictors (ii) very computationally costly. 

We estimate the generalised additive regression by backfitting (see Chapter 9 of ESL), which we explicly demonstrate below. 

In [8]:
%%time

# Degrees of freedom for the regression splines
nonlinear=['MedInc', 'HouseAge', 'AveOccup', 'AveRooms', 'Population', 'AveBedrms']
dfs=[8,8,5,3,2,6]
spline_dfs=dict(zip(nonlinear,dfs))

# Backfitting algorithm
resid1 = np.copy(y_train)  # initialising the residual 
for i in range(10):
    gam2 = generalised_additive_regression()
    gam2.fit(X_train.iloc[:,:-2], resid1, spline_dfs)
    resid2 = y_train - gam2.predict(X_train.iloc[:,:-2])
    
    gam1 = KernelReg(resid2, X_train.iloc[:,[6,7]], var_type=2*'c', reg_type='lc')
    resid1 = y_train-gam1.fit(X_train.iloc[:,[6,7]])[0]
    
    
gam2 = generalised_additive_regression()
gam2.fit(X_train.iloc[:,:-2], resid1, spline_dfs)

Wall time: 2h 48min 50s


##Model Stacking

The <TT>statlearning</TT> module provides functionality for model stacking based on linear and local regressions as meta models. 

In [9]:
from statlearning import stack_design_matrix, linear_stack, local_stack

To fit a model stack, we should ideally use cross validation predictions as features, or a validation set. It would be extremely time consuming to conduct cross validation for the GAM, so that we use fitted values for illustrative purposes only.   

In [10]:
%%time
X_train_stack= stack_design_matrix([gb], [X_train_tree], y_train, cv=10)
X2 = gam1.fit(X_train.iloc[:,[6,7]])[0] + gam2.predict(X_train)
X_train_stack = np.hstack((X_train_stack, X2.reshape((-1,1))))

Wall time: 39.9 s


We fit the model stack as follows. The <TT>normalise=True</TT> option leads to a weighted average. 

In [11]:
stack1 = linear_stack()
stack1.fit(X_train_stack, y_train, normalise=True)
stack1.beta.round(3)

array([ 0.028,  0.972])

The stack places almost all the weight on the GAM, since we "unfairly" used fitted values rather than cross validation predictions for it. We therefore change the stack to be a simple average for to compute the model evaluation results below.   

In [12]:
stack1.beta=np.array([0.5,0.5])

For illustrative purposes, we can estimate a local regression as follows.

In [13]:
%%time
stack2 = local_stack()
stack2.fit(X_train_stack, y_train)

Wall time: 7min 4s


##Model evaluation

The results show that our generalised additive model (labelled Splines + Local) outperforms boosting, and that the average of the two models outpeforms both of them by a meaningful margin. 

In [14]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE', 'Test MAE', 'SE']
rows=['OLS', 'Regression Splines', 'Local Regression', 'Decision tree', 'Bagged trees', 
      'Random forest', 'Gradient boosting', 'Splines + Local', 'Linear Stack', 'Local stack']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

methods=[ols, gam, kernel, tree, bag, rf, gb]

y_preds = np.zeros((len(y_test),len(rows)))
for i, method in enumerate(methods):
    if i < 2:
        y_preds[:,i] = method.predict(X_test)
    elif i==2:
        y_preds[:,i] = method.fit(X_test.iloc[:,[0,6,7]])[0]
    else:
        y_preds[:,i] = method.predict(X_test_tree)
        
y_preds[:,7] = gam1.fit(X_test.iloc[:,[6,7]])[0] + gam2.predict(X_test)

# This block truncates implausible predictions
y_preds[y_preds>1.609]=1.609
y_preds[y_preds<-2]=-2

# In case of numerical instability: 
y_preds[np.isnan(y_preds[:,7]), 7]=y_preds[np.isnan(y_preds[:,7]),6]

X_test_stack = y_preds[:,[6,7]]
y_preds[:,-2] = stack1.predict(X_test_stack)
y_preds[:,-1] = stack2.predict(X_test_stack)

for i in range(len(rows)):
    results.iloc[i,0], results.iloc[i,1] = rmse_jack(y_test, y_preds[:,i])
    results.iloc[i,2], results.iloc[i,3] = r2_jack(y_test, y_preds[:,i])
    results.iloc[i,4], results.iloc[i,5] = mae(y_test, y_preds[:,i])

results.round(3)

,Test RMSE,SE,Test R2,SE,Test MAE,SE
OLS,0.314,0.003,0.696,0.005,0.235,0.002
Regression Splines,0.267,0.003,0.780,0.004,0.195,0.001
Local Regression,0.276,0.003,0.764,0.005,0.201,0.001
Decision tree,0.313,0.003,0.697,0.006,0.225,0.002
Bagged trees,0.255,0.003,0.799,0.004,0.181,0.001
Random forest,0.249,0.003,0.808,0.004,0.179,0.001
Gradient boosting,0.232,0.003,0.833,0.004,0.163,0.001
Splines + Local,0.227,0.003,0.841,0.004,0.156,0.001
Linear Stack,0.214,0.003,0.858,0.004,0.148,0.001
Local stack,0.228,0.003,0.839,0.004,0.155,0.001
